In [2]:
"""Module providingFunction printing python version."""
import torch.nn as nn
import torch
import torch._inductor.config

torch._inductor.config.debug = True
torch._inductor.config.trace.output_code = True
torch._inductor.config.verbose_progress = True
torch._inductor.config.trace.graph_diagram = True
torch._inductor.config.cpp.cxx = ("g++",)

In [1]:
!pip3 install --pre torch==2.0.0.dev20230301 --index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 2.0.0.dev20230301+cpu which is incompatible.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 2.0.0.dev20230301+cpu which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 2.0.0.dev20230301+cpu which is incompatible.
fastai 2.7.11 requires torch<1.14,>=1.7, but you have torch 2.0.0.dev20230301+cpu which is incompatible.


In [ ]:
"""Module providingFunction printing python version."""
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32, 64)
        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.nn.functional.gelu(x)
        return x

model = MLP()

batch_size = 8
input = torch.randn(batch_size, 32)

In [ ]:
import torch._dynamo
from torch._functorch.aot_autograd import aot_module_simplified

def toy_backend(gm, sample_input):
    print('Dynamo produced a fx graph in Torch IR:')
    gm.print_readable()
    
    print('Notice that sample_input is a list of flattened FakeTensor:')
    print(sample_input)
    return gm.forward

torch._dynamo.reset()
fn = torch.compile(backend=toy_backend)(model)

out = fn(input)

Dynamo produced a fx graph in Torch IR:
class GraphModule(torch.nn.Module):
    def forward(self, x : torch.Tensor):
        # File: <ipython-input-3-bcf5f62b34e9>:8, code: x = self.fc1(x)
        self_fc1 = self.self_fc1(x);  x = None
        
        # File: <ipython-input-3-bcf5f62b34e9>:9, code: x = torch.nn.functional.gelu(x)
        gelu = torch._C._nn.gelu(self_fc1);  self_fc1 = None
        return (gelu,)
        
Notice that sample_input is a list of flattened FakeTensor:
[FakeTensor(FakeTensor(..., device='meta', size=(8, 32)), cpu)]


In [ ]:
import torch._dynamo
from torch._functorch.aot_autograd import aot_module_simplified

def toy_backend(gm, sample_input):
    def my_compiler(gm, sample_input):
        print('AOTAutograd produced a fx graph in Aten IR:')
        gm.print_readable()
        return gm.forward
    # Invoke AOTAutograd
    return aot_module_simplified(
        gm,
        sample_input,
        fw_compiler=my_compiler
    )

torch._dynamo.reset()
fn = torch.compile(backend=toy_backend, dynamic=True)(model)
out = fn(input)
out.sum().backward()

AOTAutograd produced a fx graph in Aten IR:
class GraphModule(torch.nn.Module):
    def forward(self, primals_1: f32[64, 32], primals_2: f32[64], primals_3: f32[s0, 32]):
        # File: <ipython-input-3-bcf5f62b34e9>:8, code: x = self.fc1(x)
        t: f32[32, 64] = torch.ops.aten.t.default(primals_1);  primals_1 = None
        addmm: f32[s0, 64] = torch.ops.aten.addmm.default(primals_2, primals_3, t);  primals_2 = t = None
        
        # File: <ipython-input-3-bcf5f62b34e9>:9, code: x = torch.nn.functional.gelu(x)
        gelu: f32[s0, 64] = torch.ops.aten.gelu.default(addmm)
        return [gelu, primals_3, addmm]
        
AOTAutograd produced a fx graph in Aten IR:
class GraphModule(torch.nn.Module):
    def forward(self, primals_3: f32[s0, 32], addmm: f32[s0, 64], tangents_1: f32[s0, 64]):
        # File: <ipython-input-3-bcf5f62b34e9>:9, code: x = torch.nn.functional.gelu(x)
        gelu_backward: f32[s0, 64] = torch.ops.aten.gelu_backward.default(tangents_1, addmm);  tangent

/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py:1251: UserWarning: Your compiler for AOTAutograd is returning a a function that doesn't take boxed arguments. Please wrap it with functorch.compile.make_boxed_func or handle the boxed arguments yourself. See https://github.com/pytorch/pytorch/pull/83137#issuecomment-1211320670 for rationale.
  warnings.warn(


In [ ]:
from torch._inductor.decomposition import decompositions as default_decompositions

decompositions = default_decompositions.copy()
default_decompositions.update(
    torch._decomp.get_decompositions([
        torch.ops.aten.addmm,
    ])
)

In [ ]:
def toy_backend(gm, sample_input):
    def my_compiler(gm, sample_input):
        print('Decomposed fx graph in Aten IR:')
        gm.print_readable()
        return gm
    # Invoke AOTAutograd
    return aot_module_simplified(
        gm,
        sample_input,
        decompositions=decompositions,
        fw_compiler=my_compiler
    )

In [ ]:
torch._dynamo.reset()
fn = torch.compile(backend=toy_backend, dynamic=True)(model)
out = fn(input)

Decomposed fx graph in Aten IR:
class GraphModule(torch.nn.Module):
    def forward(self, primals_1: f32[64, 32], primals_2: f32[64], primals_3: f32[s0, 32]):
        # File: <ipython-input-3-bcf5f62b34e9>:8, code: x = self.fc1(x)
        permute: f32[32, 64] = torch.ops.aten.permute.default(primals_1, [1, 0]);  primals_1 = None
        addmm: f32[s0, 64] = torch.ops.aten.addmm.default(primals_2, primals_3, permute);  primals_2 = permute = None
        
        # File: <ipython-input-3-bcf5f62b34e9>:9, code: x = torch.nn.functional.gelu(x)
        mul: f32[s0, 64] = torch.ops.aten.mul.Tensor(addmm, 0.5)
        mul_1: f32[s0, 64] = torch.ops.aten.mul.Tensor(addmm, 0.7071067811865476)
        erf: f32[s0, 64] = torch.ops.aten.erf.default(mul_1);  mul_1 = None
        add: f32[s0, 64] = torch.ops.aten.add.Tensor(erf, 1);  erf = None
        mul_2: f32[s0, 64] = torch.ops.aten.mul.Tensor(mul, add);  mul = add = None
        return [mul_2, primals_3, addmm]
        


/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py:1251: UserWarning: Your compiler for AOTAutograd is returning a a function that doesn't take boxed arguments. Please wrap it with functorch.compile.make_boxed_func or handle the boxed arguments yourself. See https://github.com/pytorch/pytorch/pull/83137#issuecomment-1211320670 for rationale.
  warnings.warn(


In [ ]:
prims_decomp = torch._decomp.get_decompositions([
    torch.ops.aten.add,
    torch.ops.aten.expand.default,
])

def fn(a, b):
    return a + b

def toy_backend(gm, sample_input):
    def my_compiler(gm, sample_input):
        print('Further Decomposed fx graph in Aten IR:')
        gm.print_readable()
        return gm
    # Invoke AOTAutograd
    return aot_module_simplified(
        gm,
        sample_input,
        decompositions=prims_decomp,
        fw_compiler=my_compiler
    )
    
torch._dynamo.reset()
fn = torch.compile(backend=toy_backend)(fn)
out = fn(torch.randn(3, dtype=torch.float), torch.randn(3, 3, dtype=torch.half))

Further Decomposed fx graph in Aten IR:
class <lambda>(torch.nn.Module):
    def forward(self, arg0_1: f32[3], arg1_1: f16[3, 3]):
        # File: <ipython-input-9-c65886766946>:7, code: return a + b
        _to_copy: f32[3, 3] = torch.ops.aten._to_copy.default(arg1_1, dtype = torch.float32);  arg1_1 = None
        broadcast_in_dim: f32[3, 3] = torch.ops.prims.broadcast_in_dim.default(arg0_1, [3, 3], [1]);  arg0_1 = None
        add: f32[3, 3] = torch.ops.prims.add.default(broadcast_in_dim, _to_copy);  broadcast_in_dim = _to_copy = None
        return (add,)
        


/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py:1251: UserWarning: Your compiler for AOTAutograd is returning a a function that doesn't take boxed arguments. Please wrap it with functorch.compile.make_boxed_func or handle the boxed arguments yourself. See https://github.com/pytorch/pytorch/pull/83137#issuecomment-1211320670 for rationale.
  warnings.warn(
